In [32]:
import torch as torch
from torch import nn as nn
from torch import functional as f
from torch import optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as tf
# import timm
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import pandas as pd



import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv


In [33]:
iris_datasets = datasets.load_iris()
data = iris_datasets["data"]
label = iris_datasets["target"]

In [34]:
data.dtype

dtype('float64')

In [35]:
# loading the iris dataset
iris_datasets = datasets.load_iris()
data = iris_datasets["data"]
label = iris_datasets["target"]

X_train,X_test,Y_train,Y_test = train_test_split(data,label, test_size=0.2,random_state= 42)



# preprocessing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)



# converting the data into tensors for pytorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
Y_train = torch.tensor(Y_train,dtype=torch.float32)
Y_test = torch.tensor(Y_test,dtype=torch.float32)






In [36]:

# building the model for the classification
class image_classifier(nn.Module):
  def __init__(self, input_layer=4,hidden_layer1=8,hidden_layer2=12,output_layer=3):
    super().__init__()
    self.fully_connected1 = nn.Linear(input_layer,hidden_layer1)
    self.fully_connected2 = nn.Linear(hidden_layer1,hidden_layer2)
    self.output = nn.Linear(hidden_layer2,output_layer)



  def forward(self,x):
    x = torch.relu(self.fully_connected1(x))
    x = torch.relu(self.fully_connected2(x))
    x = self.output(x)

    return x



In [37]:
# creating an instance of our model in pytorch
model = image_classifier()


In [38]:
# setting up our credentials for the training of our model
optimizer = optim.Adam(model.parameters(),lr=0.001)
evaluation = nn.CrossEntropyLoss()



In [39]:
Epoch = 100
loss = []

for epoch in range(Epoch):
  # zero the gradient
  optimizer.zero_grad()
  training  = model(X_train)
  # compute the loss
  losses = evaluation(training,Y_train.long())

  # pushing back the losses for optimization again to reduce the errors
  losses.backward()

  #  update the weiths
  optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f"[Epoch:{epoch+1}/{Epoch}], Loss: {losses.item(): .4f}")


[Epoch:10/100], Loss:  1.0805
[Epoch:20/100], Loss:  1.0561
[Epoch:30/100], Loss:  1.0288
[Epoch:40/100], Loss:  0.9973
[Epoch:50/100], Loss:  0.9605
[Epoch:60/100], Loss:  0.9156
[Epoch:70/100], Loss:  0.8647
[Epoch:80/100], Loss:  0.8125
[Epoch:90/100], Loss:  0.7638
[Epoch:100/100], Loss:  0.7211


In [40]:
# evaluating our model on the testing dataset
model.eval()
# setting up an inference mode to forget about calculating the gradient
with torch.no_grad():
  testing = model(X_test)
  _, predicted = torch.max(testing.data, 1)
  accuracy = (predicted == Y_test).sum().item() / len(Y_test) * 100
  print(f'Test Accuracy: {accuracy:.2f}%')



Test Accuracy: 93.33%


In [54]:
# we want to test ur model on new set of features
new_dataPoint = [[5.1,	3.5,	1.4,	0.2]]
new_dataPoint = torch.tensor(new_dataPoint, dtype=torch.float32)
model.eval()
with torch.inference_mode():
  output = model(new_dataPoint)

In [55]:
output

tensor([[ 0.4797, -0.4936, -0.6414]])

In [56]:
if (torch.argmax(output).item()) == 0:
  print("Setosa")
elif (torch.argmax(output).item()) == 1:
  print("Versicolor")
elif (torch.argmax(output).item()) == 2:
  print("Virginica")

else:
  print("no flower found please check your input!!!!")

Setosa
